In [1]:
open System
open System.IO
open System.Text.RegularExpressions

In [57]:
let test = 
    """Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11"""

In [30]:
type Card = {Id: int; WinningNumbers: int list; ScratchedNumbers: int list}
let createCard id w s = {Id = id; WinningNumbers = w; ScratchedNumbers = s}

In [8]:
#r "nuget:FParsec"
open FParsec

Installed Packages FParsec, 1.1.1

In [28]:

run p "Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53"

Success: 1 Item1 1 Item2 <null> Item3 (Ln: 1, Col: 9) Index 8 Line 1 Column 9 StreamName

In [29]:
let pWinningNumbers = manyTill (spaces >>. pint32 .>> spaces) (pstring "|")
let pScratchedNumbers = many (spaces >>. pint32 .>> spaces)
run (pipe3 pId pWinningNumbers pScratchedNumbers (fun x y z -> printfn "%A %A %A" x y z)) "Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53"

1 [41; 48; 83; 86; 17] [83; 86; 6; 31; 17; 9; 48; 53]


Success: () Item1 <null> Item2 <null> Item3 (Ln: 1, Col: 49) Index 48 Line 1 Column 49 StreamName

In [34]:
let pId  = pstring "Card " >>. pint32 .>> pstring ":" .>> spaces 
let pWinningNumbers = manyTill (spaces >>. pint32 .>> spaces) (pstring "|")
let pScratchedNumbers = many (spaces >>. pint32 .>> spaces)
let lineToCard (line:string) = 
    match run (pipe3 pId pWinningNumbers pScratchedNumbers createCard) line with 
    | Success (card, _, _) -> card 
    | _ -> failwith $"Problems with: {line}"

In [51]:
let points (card : Card) = 
    card.ScratchedNumbers
    |> List.filter (fun x -> List.contains x card.WinningNumbers)
    |> List.length
    |> (+) (-1)
    |> pown 2

In [59]:
test.Split("\r\n")
|> Array.sumBy (lineToCard >> points)

13

In [60]:
File.ReadAllLines("input_4.txt")
|> Array.sumBy (lineToCard >> points)

Error: System.IO.FileNotFoundException: Could not find file 'c:\AAPersonal\Programming\AdventOfCode2023\input_4.txt'.
File name: 'c:\AAPersonal\Programming\AdventOfCode2023\input_4.txt'
   at Microsoft.Win32.SafeHandles.SafeFileHandle.CreateFile(String fullPath, FileMode mode, FileAccess access, FileShare share, FileOptions options)
   at Microsoft.Win32.SafeHandles.SafeFileHandle.Open(String fullPath, FileMode mode, FileAccess access, FileShare share, FileOptions options, Int64 preallocationSize, Nullable`1 unixCreateMode)
   at System.IO.Strategies.OSFileStreamStrategy..ctor(String path, FileMode mode, FileAccess access, FileShare share, FileOptions options, Int64 preallocationSize, Nullable`1 unixCreateMode)
   at System.IO.Strategies.FileStreamHelpers.ChooseStrategyCore(String path, FileMode mode, FileAccess access, FileShare share, FileOptions options, Int64 preallocationSize, Nullable`1 unixCreateMode)
   at System.IO.StreamReader.ValidateArgsAndOpenPath(String path, Encodi